# Wind Rose

The notebook is inspired by [this plot](https://mesonet.cdn.columbiascanner.org/onsite/windrose/IA_ASOS/PEA/PEA_yearly.png).

In [1]:
%useLatestDescriptors
%use lets-plot
%use dataframe

In [2]:
fun getData(): DataFrame<*> {
    val bins = listOf(0, 5, 7, 10, 15, 20)
    val df = DataFrame.readCSV("https://raw.githubusercontent.com/JetBrains/lets-plot-docs/master/data/pea_wind.csv")
        .drop {
            try {
                "drct"<String>().toDouble()
                "sped"<String>().toDouble()
                false
            } catch (_: Exception) {
                true
            }
        }
        .convert { "drct"<String>() }.to<Double>()
        .convert { "sped"<String>() }.to<Double>()
        .filter { "sped"<Double>() >= 2 }
        .add("speed_group") { bins.indexOfLast { it < "sped"<Double>() } }
        .remove { "valid"<kotlinx.datetime.LocalDateTime>() }
    return listOf(
        df.filter { "drct"<Double>() == 360.0 }.update { "drct"<Double>() }.with { 0.0 },
        df
    ).concat()
}

In [3]:
val df = getData()
println("Number of rows: ${df.rowsCount()}\nNumber of columns: ${df.columnsCount()}")
df.head()

Number of rows: 1005457
Number of columns: 4


station,drct,sped,speed_group
PEA,0.000000,8.050000,2
PEA,0.000000,8.050000,2
PEA,0.000000,5.750000,1
PEA,0.000000,5.750000,1
PEA,0.000000,5.750000,1


In [4]:
letsPlot(df.toMap()) +
    geomDensity(
        position = positionStack(), size = 0,
        tooltips = layerTooltips().format("^x", "{d}°")
    ) { x = "drct"; fill = asDiscrete("speed_group", order = 1) } +
    scaleXContinuous(labels = mapOf(45 to "NE", 90 to "E", 135 to "SE", 180 to "S", 225 to "SW", 270 to "W", 315 to "NW", 360 to "N")) +
    scaleFillManual(
        name = "Wind speed (mph):",
        values = listOf("#002bff", "#03d3f8", "#7afe81", "#fde609", "#ff4404", "#780200"),
        labels = mapOf(0 to "2 - 4.9", 1 to "5 - 6.9", 2 to "7 - 9.9", 3 to "10 - 14.9", 4 to "15 - 19.9", 5 to "20+")
    ) +
    labs(
        title = "Wind rose for [PEA] PELLA",
        subtitle = "Observations from 2002 to 2024",
        caption = "Data is provided by the Iowa Environmental Mesonet of Iowa State University"
    ) +
    coordPolar(xlim = Pair(0, 360), ylim = Pair(-.002, null)) +
    ggsize(800, 800) +
    themeMinimal2() +
    theme(
        plotTitle = elementText(size = 24, face = "bold"),
        plotSubtitle = elementText(size = 18),
        panelGridMinorX = elementLine(),
        panelGrid = elementLine(color = "#a0a0a0"),
        axisTicksY = elementLine(),
        axisTextX = elementText(size = 18),
        axisTitle = elementBlank()
    )